# Download Permafrost Data

The [Global Terrestrial Network for Permafrost - Database](http://www.gtnpdatabase.org/) contains a repository on data from permafrost sites. Data for both
 * Thaw Depths (aka Active Layer Thickness)
 * Temperatures
are available.

Unfortunately, the site focuses on making the data available individually involving a lot of user interaction. For analyses at scale, this proves inefficient. This notebook downloads the data locally so you can analyse the data yourself.

We are trying to build a simplified version of the GTN-P data with as few tables as possible. With the source using normalized tables, consistency of GTN-P is generally good, so, for the sake of data science ease of use, we use "fat" tables.

![Data Model](Data_Model.png)


## Packages

There are a few packages which may not be part of a default installation. Also, some may require an installation at the OS level outside the realm of pipenv, conda, or pip.
* [geopandas](https://geopandas.org/) is required to save data with geospatial information
* [PostgreSQL](https://www.postgresql.org/) and its Geospatial Information System extension [PostGIS](https://postgis.net/install/) are required locally. We will try to make a docker package available. Note that PostGIS needs to be enabled per-database. For Mac users, consider [Postgret.app](https://postgresapp.com/), for Windows users, consider [PostgreSQL Portable](https://github.com/garethflowers/postgresql-portable)
* [pycountry](https://pypi.org/project/pycountry/) is used to standardise country names and look up their ISO codes
* [Selenium](https://pypi.org/project/selenium/) is required to load pages that use JavaScript to build their content. This generally requires some installation on the OS as Selenium needs a working web browser to render these pages.
* [tqdm](https://github.com/tqdm/tqdm) for progress bars (although we could also use ipywidgets progress bar, but tqdm also measures and displays time per iteration and often ETA)
* We also use sqlalchemy, pandas, numpy, and scipy. To avoid data conversion difficulties, we store local data files as binary files, preference being parquet, so pyarrow or fastparquet may be required.

In [204]:
import io
import pandas as pd
import geopandas as gpd
import shapely
from bs4 import BeautifulSoup
import bs4
import requests
import zipfile
from pathlib import Path
import time
from tqdm import tqdm
import numpy as np
import pycountry
import glob
import os
import re
from selenium import webdriver
from sqlalchemy import create_engine
import sqlalchemy
from tqdm import tqdm
import json
import datetime
import socket
import urllib
import sys
from scipy.interpolate import interp1d

## Web Crawling using Selenium

The web page is dynamically generated using JavaScript. This makes access the content slightly more challenging. Selenium is a toolset that is created to allow for automated UI testing, but it is also used to crawl websites. 

Depending on your jupyter setup, selenium may be very tricky, or impossible to install, as it actually requires a functioning web browser to be installed on the machine that hosts the jupyter evironment. This is very often not the case for servers or cloud based installations. Should this be the case:
* Open [http://www.gtnpdatabase.org/activelayers](http://www.gtnpdatabase.org/activelayers) in a browser on your workstation
* Try to File->Save the webpage locally
* There will be a hook for you to copy and paste the file further down

In [3]:
options = webdriver.ChromeOptions() # https://stackoverflow.com/questions/50642308
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome(options=options)

If you cannot get Selenium installed or working, open [http://www.gtnpdatabase.org/activelayers](http://www.gtnpdatabase.org/activelayers) and save it as `./sun/activelayers/index.html`. Same for [http://www.gtnpdatabase.org/boreholes](http://www.gtnpdatabase.org/boreholes) and `./sun/boreholes/index.html`.

In [6]:
Path("./sun/activelayers").mkdir(exist_ok=True,parents=True)

if not os.path.exists("./sun/activelayers/index.html"):
    wd.get("http://www.gtnpdatabase.org/activelayers")
    print("waiting 10 seconds for the JavaScript to complete",end=".")
    time.sleep(10) # wait for javascript to finish
    print("...done, next page")
    activelayers_page = wd.page_source
    with open("./sun/activelayers/index.html","w+") as outfile:
        outfile.write(activelayers_page)
else:
    with open("./sun/activelayers/index.html","r") as infile:
        activelayers_page = infile.read()

Path("./sun/boreholes").mkdir(exist_ok=True,parents=True)

if not os.path.exists("./sun/boreholes/index.html"):
    wd.get("http://www.gtnpdatabase.org/boreholes")
    print("waiting 10 seconds for the JavaScript to complete",end=".")
    time.sleep(10) # wait for javascript to finish
    print("...finished")

    boreholes_temps_page = wd.page_source
    with open("./sun/boreholes/index.html","w+") as outfile:
        outfile.write(boreholes_temps_page)
else:
    with open("./sun/boreholes/index.html","r") as infile:
        boreholes_temps_page = infile.read()

Unfortunately, the usual way to parse the table using pandas´ `read_html` results in a missing entry, Station _Da Xi Gou  (Glacier Station)_  is not always added. We will use that approach further down in this notebook.

In [7]:
df = pd.read_html(wd.page_source)[0] # little known fact, pandas parses tables quite well, note it returns an array
df.columns = [c.lower() for c in df.columns] 
df[df["gtn-p"]=="CN9"]

,name,site,country,gtn-p,vegetation,permafrost,elevation,depth,data,select


### Helper Function to Log Data Issues

As we walk across the data, we will encounter deficiencies. We will set up a log database so that, if possible, deficiencies can be rectified.

In [8]:
def static_vars(**kwargs):
    def decorate(func):
        for k in kwargs:
            setattr(func, k, kwargs[k])
        return func
    return decorate

@static_vars(log=[])
def log_finding(finding):
    finding["datetime_date"] = "{:%Y-%m-%d %H:%M:%S}".format(datetime.datetime.now())
    finding["machine"] = socket.gethostname()
    log_finding.log.append(finding)
    with open("findings.json","w+t") as logfile:
        json.dump(log_finding.log,logfile)

## Database connection

If you plan to put code of yours onto public platforms such as github or gitlab, never, ever, add credentials to your source code. In fact, never ever, even if you have no such plans. We read the credentials and database server from a file names .env, in this repo, copy .env.example and edit the content so it suits your needs. We need a string in the form 

```
postgresql://<user>:<password>@<hostname>/<database>
```

in that file. Often, this is `postgresql://postgres@localhost/postgres` by default.

In [9]:
with open(".env","rt") as idfile:
    connect_string = idfile.read().strip()
conn = create_engine(connect_string)

## Web Page parsing using BeautifulSoup

The download links use an id of the site in their URL. The id field, however, is not visible on the web page, but is in the hyperlink `href` field of the table. These links are contained in c table cell (`<td>`) with an attribute `data-th="Name"`.

The html code looks like

```
<td data-th="Name"><a class="ng-binding" href="/activelayers/view/195/">Allakaket</a></td>
```

and we are after the 195, which is the last forward-slash separated field of the URL. Also, as we already use BeautifulSoup, we take this approach to populate the data frame.

### Activelayers

In [10]:
soup = BeautifulSoup(io.StringIO(activelayers_page))

entries = []
for tr in soup.findAll("tr"):
    entry = {}
    for td in tr.findAll("td"):
        if td["data-th"] == "Gtn-P":
            entry["code"] = td.text
        else:
            entry[td["data-th"].lower()] = td.text
        if td["data-th"] == "Name":
            entry["subsite_id"] = td.contents[0]["href"].split("/")[-2]
            entry["view_url"] = td.contents[0]["href"]
        elif td["data-th"] == "Site":
            entry["site_id"] = td.contents[0]["href"].split("/")[-2]
            entry["site_url"] = td.contents[0]["href"]
    if len(entry)>0:
        entries.append(entry)
dfActiveLayersCatalogue = pd.DataFrame(entries)
dfActiveLayersCatalogue.elevation = pd.to_numeric(dfActiveLayersCatalogue.elevation)
print("{} Entries loaded".format(len(dfActiveLayersCatalogue)))

253 Entries loaded


Add standard country codes and names using `pycountry`.

In [11]:
dfActiveLayersCatalogue["iso2"] = ""
dfActiveLayersCatalogue["iso3"] = ""
dfActiveLayersCatalogue["country_official_name"] = ""
for i,r in dfActiveLayersCatalogue.iterrows():
    info = pycountry.countries.search_fuzzy(r.country)
    dfActiveLayersCatalogue.at[i,"iso2"] = info[0].alpha_2
    dfActiveLayersCatalogue.at[i,"iso3"] = info[0].alpha_3
    if "country_official_name" in info[0].__dict__.keys():
        dfActiveLayersCatalogue.at[i,"country_official_name"] = info[0].country_official_name
    else:
        dfActiveLayersCatalogue.at[i,"country_official_name"] = info[0].name
dfActiveLayersCatalogue["subsite_type"] = "activelayer"
dfActiveLayersCatalogue.head()

,name,subsite_id,view_url,site,site_id,site_url,country,type,code,vegetation,permafrost,elevation,data,select,iso2,iso3,country_official_name,subsite_type
0,56 Mile,227,/activelayers/view/227/,Franklin Bluff,16,/sites/view/16/,United States,Grid,U31 A,Tundra,Continuous,114.0,No,,US,USA,United States,activelayer
1,Abisko,42,/activelayers/view/42/,Abisko,6,/sites/view/6/,Sweden,Grid,S2,Forest Tundra,Discontinuous,507.0,No,,SE,SWE,Sweden,activelayer
2,Akhmelo Channel,88,/activelayers/view/88/,Cherskii,8,/sites/view/8/,Russia,Grid,R17,Forest Tundra,Continuous,5.0,Yes,,RU,RUS,Russian Federation,activelayer
3,Alazeya River,82,/activelayers/view/82/,Cherskii,8,/sites/view/8/,Russia,Grid,R22,Shrub Tundra,Continuous,60.0,Yes,,RU,RUS,Russian Federation,activelayer
4,Alexandria Fiord,27,/activelayers/view/27/,Alexandria Fiord,329,/sites/view/329/,Canada,Grid,C1,Tundra,Continuous,0.0,Yes,,CA,CAN,Canada,activelayer


### Temperatures

In [12]:
soup = BeautifulSoup(io.StringIO(boreholes_temps_page), 'lxml')

entries = []
for tr in soup.findAll("tr"):
    entry = {}
    for td in tr.findAll("td"):
        if td["data-th"] == "Gtn-P":
            entry["code"] = td.text
        else:
            entry[td["data-th"].lower()] = td.text
        if td["data-th"] == "Name":
            entry["subsite_id"] = td.contents[0]["href"].split("/")[-2]
            entry["view_url"] = td.contents[0]["href"]
        elif td["data-th"] == "Site":
            entry["site_id"] = td.contents[0]["href"].split("/")[-2]
            entry["site_url"] = td.contents[0]["href"]
    if len(entry)>0:
        entries.append(entry)
dfBoreholeTempsCatalogue = pd.DataFrame(entries)
dfBoreholeTempsCatalogue.elevation = pd.to_numeric(dfBoreholeTempsCatalogue.elevation)
print("{} Entries loaded".format(len(dfBoreholeTempsCatalogue)))

1380 Entries loaded


There is a typo in the country names, which we want to log.

In [13]:
dfBoreholeTempsCatalogue["iso2"] = ""
dfBoreholeTempsCatalogue["iso3"] = ""
dfBoreholeTempsCatalogue["country_official_name"] = ""
for i,r in dfBoreholeTempsCatalogue.iterrows():
    if r.country.lower() == "kyrgystan":
        log_finding({"topic":"borehole temperatures",
                    "site_id":r.site_id,
                    "subsite_id":r.subsite_id,
                    "site_url":r.site_url,
                    "site":r.site,
                    "diagnosis":"Country spelling incorrect, '{}' is spelled 'Kyrgyzstan'".format(r.country),
                    "fix":"changed to 'Kyrgyzstan'",
                    "needs_attention":False})
        info = pycountry.countries.search_fuzzy("Kyrgyzstan")
    else:
        info = pycountry.countries.search_fuzzy(r.country)
    dfBoreholeTempsCatalogue.at[i,"iso2"] = info[0].alpha_2
    dfBoreholeTempsCatalogue.at[i,"iso3"] = info[0].alpha_3
    if "country_official_name" in info[0].__dict__.keys():
        dfBoreholeTempsCatalogue.at[i,"country_official_name"] = info[0].country_official_name
    else:
        dfBoreholeTempsCatalogue.at[i,"country_official_name"] = info[0].name
        
dfBoreholeTempsCatalogue["subsite_type"] = "temperatures"
dfBoreholeTempsCatalogue.head()

,name,subsite_id,view_url,site,site_id,site_url,country,code,vegetation,permafrost,elevation,depth,data,select,iso2,iso3,country_official_name,subsite_type
0,0 (Deputatsky),1007,/boreholes/view/1007/,Deputatskiy,331,/sites/view/331/,Russia,RU 118,Shrub Tundra,Continuous,462.0,88.00,No,,RU,RUS,Russian Federation,temperatures
1,01TC1,1744,/boreholes/view/1744/,Yukon,73,/sites/view/73/,Canada,CA 196,Grassland,Continuous,18.0,8.00,No,,CA,CAN,Canada,temperatures
2,01TC2,1745,/boreholes/view/1745/,Yukon,73,/sites/view/73/,Canada,CA 197,Grassland,Continuous,95.0,10.00,No,,CA,CAN,Canada,temperatures
3,03TC1,1743,/boreholes/view/1743/,Yukon,73,/sites/view/73/,Canada,CA 195,Grassland,Discontinuous,3.0,6.00,No,,CA,CAN,Canada,temperatures
4,08 (Deputatsky),1008,/boreholes/view/1008/,Deputatskiy,331,/sites/view/331/,Russia,RU 119,Forest Tundra,Continuous,473.0,96.00,No,,RU,RUS,Russian Federation,temperatures


### Combine Borehole Depth and Temperatures Catalogues into one

There is historic reasons these were created separately. Our aim is to assess all permafrost data, so we want one catalogue.

In [17]:
dfBoreholeTempsCatalogue["type"] = "N/A" # pad a missing field which is only present in the borhole data
dfSites = dfActiveLayersCatalogue.append(dfBoreholeTempsCatalogue).reset_index()
# convert Yes/No text into boolean
dfSites["have_data"] = False
idx = dfSites[dfSites["data"] == "Yes"].index
dfSites.at[idx,"have_data"] = True
del dfSites["data"]
dfSites.rename(columns={"name":"subsite_name","site":"site_name"},inplace=True)
dfSites

,index,name,subsite_id,view_url,site,site_id,site_url,country,type,code,vegetation,permafrost,elevation,select,iso2,iso3,country_official_name,subsite_type,depth,have_data
0,0,56 Mile,227,/activelayers/view/227/,Franklin Bluff,16,/sites/view/16/,United States,Grid,U31 A,Tundra,Continuous,114.0,,US,USA,United States,activelayer,NaN,False
1,1,Abisko,42,/activelayers/view/42/,Abisko,6,/sites/view/6/,Sweden,Grid,S2,Forest Tundra,Discontinuous,507.0,,SE,SWE,Sweden,activelayer,NaN,False
2,2,Akhmelo Channel,88,/activelayers/view/88/,Cherskii,8,/sites/view/8/,Russia,Grid,R17,Forest Tundra,Continuous,5.0,,RU,RUS,Russian Federation,activelayer,NaN,True
3,3,Alazeya River,82,/activelayers/view/82/,Cherskii,8,/sites/view/8/,Russia,Grid,R22,Shrub Tundra,Continuous,60.0,,RU,RUS,Russian Federation,activelayer,NaN,True
4,4,Alexandria Fiord,27,/activelayers/view/27/,Alexandria Fiord,329,/sites/view/329/,Canada,Grid,C1,Tundra,Continuous,0.0,,CA,CAN,Canada,activelayer,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628,1375,Zima railst,1677,/boreholes/view/1677/,Irkutsk,369,/sites/view/369/,Russia,N/A,,Other,Isolated patches,456.0,,RU,RUS,Russian Federation,temperatures,3.00,True
1629,1376,Zugspitze 01/07 (W),859,/boreholes/view/859/,Zugspitze,263,/sites/view/263/,Germany,N/A,DE 01,Other,Isolated Patches,2922.0,,DE,DEU,Germany,temperatures,44.00,False
1630,1377,Zugspitze 02/07 (E),858,/boreholes/view/858/,Zugspitze,263,/sites/view/263/,Germany,N/A,DE 02,Other,Isolated Patches,2922.0,,DE,DEU,Germany,temperatures,58.00,False
1631,1378,Zugspitze tunnel,1798,/boreholes/view/1798/,Zugspitze,263,/sites/view/263/,Germany,N/A,,No Vegetation,Mountain,2785.0,,DE,DEU,Germany,temperatures,50.00,False


In [18]:
Path("./sun/catalogue").mkdir(exist_ok=True,parents=True)
dfSites.to_parquet("./sun/catalogue/sites.catalogue.parquet")

In [19]:
#dfSites = pd.read_parquet("./sun/catalogue/sites.catalogue.parquet")

## Download Site Metadata

Each site has one landing page which describes more about its location and some subsites. In particular, we are after Observation Type, Area, Latitude/Longitude, which is not part of the previous dataset.

Note that we have a hybrid of site and subsite data. In particular, Latitude and Longitude are site data, while name is the subsite name.

As all data we want are inside html tables, we use pandas' `read_html`, which returns a list of dataframes.

In [25]:
nok = []
alldata = []

site_urls = dfSites.site_url.unique()

for site_url in tqdm(site_urls):
    Path("./sun"+site_url).mkdir(exist_ok=True,parents=True)
    if not os.path.exists("./sun"+site_url+"index.html"):
        r = requests.get("http://www.gtnpdatabase.org{}".format(site_url))
        if r.ok:
            with open("./sun"+site_url+"index.html","w+") as outfile:
                outfile.write(r.text)
        else:
            nok.append("http://www.gtnpdatabase.org{}".format(site_url))
            continue
    dfGeneral,dfSiteCentre,dfBBox = pd.read_html("./sun"+site_url+"index.html")
    alldata.append({"site_id":site_url.split("/")[-2],
                   "observation_type":dfGeneral.iloc[1].values[1],
                  "area":pd.to_numeric(dfGeneral.iloc[2].values[1].replace(",","").split(" ")[0]),
                  "site_lon":pd.to_numeric(dfSiteCentre.iloc[0].values[1].replace("°","")),
                  "site_lat":pd.to_numeric(dfSiteCentre.iloc[1].values[1].replace("°","")),
                  "bbox_lat_max":pd.to_numeric(dfBBox.iloc[0].values[1].replace("°","").split(":")[1]),
                  "bbox_lat_min":pd.to_numeric(dfBBox.iloc[2].values[1].replace("°","").split(":")[1]),
                  "bbox_lon_min":pd.to_numeric(dfBBox.iloc[1].values[0].replace("°","").split(":")[1]),
                  "bbox_lon_max":pd.to_numeric(dfBBox.iloc[1].values[2].replace("°","").split(":")[1])})
    time.sleep(.25)

if len(nok)>0:
    print("Warning, had problems downloading {}".format(nok))
else:
    print("Donwload complete")
dfSiteInfo = pd.DataFrame(alldata)
dfSiteInfo.observation_type = dfSiteInfo.observation_type.replace(np.nan, '', regex=True)

100%|██████████| 352/352 [01:31<00:00,  3.84it/s]

Donwload complete


In [27]:
dfSites = dfSites.merge(dfSiteInfo,on="site_id")
Path("./mercury/catalogue").mkdir(exist_ok=True,parents=True)
dfSites.to_parquet("./mercury/catalogue/sites.catalogue.parquet")

In [28]:
gdfSites = gpd.GeoDataFrame(dfSites, geometry=gpd.points_from_xy(dfSites.site_lon,dfSites.site_lat)).set_crs(epsg=4326)
for c in ["subsite_id","site_id","elevation","depth","area","site_lat","site_lon","bbox_lat_max","bbox_lat_min","bbox_lon_min","bbox_lon_max"]:
    gdfSites[c] = pd.to_numeric(gdfSites[c])
del gdfSites["index"]
del gdfSites["select"]

In [29]:
gdfSites.to_postgis("t_permafrost_sites",con=conn,dtype={"subsite_name":sqlalchemy.types.VARCHAR(100),
                                                         "subsite_id":sqlalchemy.types.INT,
                                                         "view_url":sqlalchemy.types.VARCHAR(100),
                                                         "site_name":sqlalchemy.types.VARCHAR(100),
                                                         "site_id":sqlalchemy.types.INT,
                                                         "site_url":sqlalchemy.types.VARCHAR(100),
                                                         "country":sqlalchemy.types.VARCHAR(40),
                                                         "type":sqlalchemy.types.VARCHAR(20),
                                                         "code":sqlalchemy.types.VARCHAR(20),
                                                         "vegetation":sqlalchemy.types.VARCHAR(50),
                                                         "permafrost":sqlalchemy.types.VARCHAR(50),
                                                         "iso2":sqlalchemy.types.VARCHAR(2),
                                                         "iso3":sqlalchemy.types.VARCHAR(3),
                                                         "country_official_name":sqlalchemy.types.VARCHAR(80),
                                                         "subsite_type":sqlalchemy.types.VARCHAR(20),
                                                         "observation_type":sqlalchemy.types.VARCHAR(20),
                                                        },if_exists="replace")

In [30]:
#gdfSites = gpd.read_postgis("SELECT * FROM t_permafrost_sites",con=conn,geom_col="geometry")

## Subsite Metadata

Similar to sites, which are a collection of one or more subsites (which actually resemble the borholes, or borehole arrays), subsites have their own metadata site. Metadata there varies between activelayer and temperatures subsites, also, some tables are not present in the html code if no data are associated with them.

In [408]:
def get_dates_from_soup(soup):
    dates = {}
    for e in soup.find("fieldset").findAll("small"):
        fields = e.text.split(":")
        key = fields[0].lower()
        value = pd.to_datetime(fields[1])
        dates[key] = value
    return dates

def get_explanation_from_soup(soup):
    text = []
    for p in soup.find("div",{"class":"leftCol"}).findAll("p",{"class":""}):
        if p.find("strong"):
            header = p.find("strong")
            text.append("# "+header.text)
        elif p.find("a"):
            link = p.find("a",href=True)
            text.append("[{}]({})".format(link.text,link["href"]))
        else:
            text.append(p.text.strip())
    return ("\n\n".join(text))

def get_references_from_soup(soup,site_id,conn,DEFER=True):
    references = []
    entries = []
    el = soup.find('hr')
    while(el):
        el = el.next_sibling
        if isinstance(el, bs4.element.NavigableString):
            if len(el.strip())>0:
                entries.append(el.strip()) 

    for entry in entries:
        try:
            dfExisting = pd.read_sql("SELECT * FROM t_references WHERE reference='{}'".format(
                entry.replace("'","''")),con=conn) # need this as we use a single ' to quote a string
            if len(dfExisting)>0:
                references.extend(dfExisting.doi.values)
                continue
        except:
            print(entry,end="\n")
            pass
        
        url = "https://api.crossref.org/works?query={}".format(urllib.parse.quote(entry))
        if DEFER:
            with open("deferred.queries.txt","a+t") as todolist:
                todolist.write(url+"\n")
            continue
        r = requests.get(url,headers=headers)
        record = {"doi":"","reference":entry,"site_id":site_id}
        if r.ok:
            result = json.loads(r.text)
            item = result["message"]["items"][0]
            if "URL" in item.keys():
                new_record = {"doi":item["URL"],"reference":entry,"result":json.dumps(item)}
        else:
            continue
            
        if "x-rate-limit-limit" in r.headers.keys():
            rate_limit = r.headers["x-rate-limit-limit"]
            rate_interval = pd.to_numeric(r.headers["x-rate-limit-interval"].replace("s",""))
            try:
                sleep = rate_interval/rate_limit
            except:
                sleep = 0.1
        time.sleep(sleep)
        references.append(new_record["doi"])
        xdf = pd.DataFrame(new_record,index=[0])
        xdf.to_sql("t_references",con=conn,dtype={"doi":sqlalchemy.types.VARCHAR(100),
                                   "reference":sqlalchemy.types.VARCHAR(1000),
                                   "result":sqlalchemy.types.VARCHAR(50000)},
                   if_exists="append",index=False)
    return entries,references

In [409]:
keep_columns = {'CALM-Code:':"calm_code", 'Drilling Angle:':"drilling_angle", 
                'Drilling Method:':"drilling_method", 'Elevation:':"subsite_elevation", 
                'GTN-P:':"gtn_p", 'Latitude:':"subsite_lat", 'Longitude:':"subsite_lon",
                'Lithology:':"lithology", 'Morphology:':"morphology", 
                'Responsible Countries:':"responsible_countries",
                'Responsible Person:':"responsible_person"}

In [534]:
nok = []
manual_review = []

adfSubsiteInfo = []

for i,row in tqdm(gdfSites.iterrows()):
    targetfolder = "./sun"+row.view_url
    targetfile = os.path.join(targetfolder,"index.html")
    if os.path.exists(targetfile):
        with open(targetfile) as infile:
            content = infile.read()
    else:
        Path(targetfolder).mkdir(exist_ok=True,parents=True)
        r = requests.get("http://www.gtnpdatabase.org{}".format(row.view_url)) #"/boreholes/view/1097/")) #
        if r.ok:
            content = r.text
        else:
            nok.append(row.view_url)
            continue
        with open(targetfile,"w+") as outfile:
            outfile.write(content)

    soup = BeautifulSoup(content)
    
    df = pd.DataFrame()
    tabs = {0:"general"}
    for infoTable in soup.findAll("table",{"class":"infoTable"}):
        try:
            df = pd.read_html(io.StringIO(str(infoTable)))[0]
        except:
            continue
    dates = get_dates_from_soup(soup)
    dfArray = pd.read_html(io.StringIO(content))

    # we dont try to parse all information...
    #for i in range(len(dfArray)):
    #    dfArray[i]["array_no"] = i

    ddf = pd.DataFrame().append(dfArray)
    ddf = ddf[ddf[0].isin(keep_columns.keys())][[0,1]]
    ddf = ddf.append(pd.DataFrame({0:["subsite_type","created","modified","subsite_id","view_url"],
                                   1:[row.subsite_type,dates["created"],dates["modified"],row.subsite_id,row.view_url]}))

    if ddf.duplicated(subset=[0]).any():
        print("warning")
        break

    if soup.find("div",{"class":"references"}):
        entries,references = get_references_from_soup(soup.find("div",{"class":"references"}),
                                                  row.site_id,conn,DEFER=False)
    else:
        entries = []
        references = []
    
    ddf = ddf.append(pd.DataFrame({0:["entries","references"],
                                   1:[json.dumps(entries),json.dumps(references)]}))
    ddf.index = ddf[0]
    del ddf[0]
    ddf = ddf.transpose()
    adfSubsiteInfo.append(ddf.rename(columns=keep_columns))
    
dfSubsites = pd.DataFrame().append(adfSubsiteInfo)

1633it [00:41, 39.06it/s]


In [537]:
dfSubsites.subsite_lon = pd.to_numeric(dfSubsites.subsite_lon.str.replace("°",""))
dfSubsites.subsite_lat = pd.to_numeric(dfSubsites.subsite_lat.str.replace("°",""))
dfSubsites.drilling_angle = pd.to_numeric(dfSubsites.drilling_angle.str.replace("°",""))
dfSubsites.subsite_elevation = pd.to_numeric(dfSubsites.subsite_elevation.str.replace("m",""))
dfSubsites.subsite_id = pd.to_numeric(dfSubsites.subsite_id).astype(int)
for c in ["calm_code","responsible_countries","responsible_person","lithology",
          "subsite_type","gtn_p","drilling_method","morphology"]:
    dfSubsites[c] = dfSubsites[c].replace(np.nan, '', regex=True)
dfSubsites.head()

,calm_code,responsible_countries,responsible_person,lithology,subsite_lon,subsite_lat,subsite_elevation,subsite_type,created,modified,subsite_id,view_url,entries,references,gtn_p,drilling_angle,drilling_method,morphology
1,U31 A,"United States,",Nikolay Shiklomanov,Organic Layer thikness:NA; mineral texture -- ...,-148.682100,69.696900,114.259277,activelayer,2014-02-03 11:00:00,2016-10-24 12:00:00,227,/activelayers/view/227/,"[""Walker, D.A., Auerbach, N.A., Bockheim, J.G....","[""http://dx.doi.org/10.1038/28839"", ""http://dx...",,NaN,,
1,U8,"United States,",Vladimir E. Romanovsky,The organic layer thickness is 0.23 m,-148.716667,69.683333,120.003014,activelayer,2013-12-27 15:00:00,2014-02-03 14:00:00,15,/activelayers/view/15/,"[""Nicolsky, D. J., Romanovsky, V. E., Alexeev,...","[""http://dx.doi.org/10.1029/2007gl029525"", ""ht...",,NaN,,
1,,"United States,",Gary D. Clow,,-146.338233,69.605883,269.993164,temperatures,2014-08-20 11:00:00,2016-06-02 16:00:00,1128,/boreholes/view/1128/,[],[],US 94,90.0,,
1,,"United States,",Doug L. Kane,,-148.750000,69.833333,76.467224,temperatures,2015-10-06 15:00:00,2015-10-06 15:00:00,1173,/boreholes/view/1173/,[],[],US 43,90.0,,
1,,"United States,",Vladimir E. Romanovsky,,-148.720766,69.674140,122.868607,temperatures,2013-12-09 14:00:00,2014-02-27 10:00:00,103,/boreholes/view/103/,[],[],,90.0,,


In [538]:
gdfSubites = gpd.GeoDataFrame(dfSubsites, 
                              geometry=gpd.points_from_xy(dfSubsites.subsite_lon,
                                                          dfSubsites.subsite_lat)).set_crs(epsg=4326)
gdfSubites

,calm_code,responsible_countries,responsible_person,lithology,subsite_lon,subsite_lat,subsite_elevation,subsite_type,created,modified,subsite_id,view_url,entries,references,gtn_p,drilling_angle,drilling_method,morphology,geometry
1,U31 A,"United States,",Nikolay Shiklomanov,Organic Layer thikness:NA; mineral texture -- ...,-148.682100,69.696900,114.259277,activelayer,2014-02-03 11:00:00,2016-10-24 12:00:00,227,/activelayers/view/227/,"[""Walker, D.A., Auerbach, N.A., Bockheim, J.G....","[""http://dx.doi.org/10.1038/28839"", ""http://dx...",,NaN,,,POINT (-148.68210 69.69690)
1,U8,"United States,",Vladimir E. Romanovsky,The organic layer thickness is 0.23 m,-148.716667,69.683333,120.003014,activelayer,2013-12-27 15:00:00,2014-02-03 14:00:00,15,/activelayers/view/15/,"[""Nicolsky, D. J., Romanovsky, V. E., Alexeev,...","[""http://dx.doi.org/10.1029/2007gl029525"", ""ht...",,NaN,,,POINT (-148.71667 69.68333)
1,,"United States,",Gary D. Clow,,-146.338233,69.605883,269.993164,temperatures,2014-08-20 11:00:00,2016-06-02 16:00:00,1128,/boreholes/view/1128/,[],[],US 94,90.0,,,POINT (-146.33823 69.60588)
1,,"United States,",Doug L. Kane,,-148.750000,69.833333,76.467224,temperatures,2015-10-06 15:00:00,2015-10-06 15:00:00,1173,/boreholes/view/1173/,[],[],US 43,90.0,,,POINT (-148.75000 69.83333)
1,,"United States,",Vladimir E. Romanovsky,,-148.720766,69.674140,122.868607,temperatures,2013-12-09 14:00:00,2014-02-27 10:00:00,103,/boreholes/view/103/,[],[],,90.0,,,POINT (-148.72077 69.67414)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,,"United States,",Kenji Yoshikawa,,-163.413000,64.682000,14.000000,temperatures,2014-01-20 13:00:00,2014-06-06 09:00:00,771,/boreholes/view/771/,[],[],US O-94,90.0,,,POINT (-163.41300 64.68200)
1,,"Italy,",Mauro Guglielmin,,10.308250,46.392250,2570.000000,temperatures,2014-01-22 10:00:00,2014-01-22 10:00:00,895,/boreholes/view/895/,[],[],IT 02,90.0,,,POINT (10.30825 46.39225)
1,,"Germany,",Andreas Poschinger,,10.987108,47.421789,2922.000000,temperatures,2014-01-21 15:00:00,2016-10-22 09:00:00,859,/boreholes/view/859/,[],[],DE 01,90.0,,,POINT (10.98711 47.42179)
1,,"Germany,",Andreas Poschinger,,10.987142,47.421778,2922.000000,temperatures,2014-01-21 15:00:00,2016-10-22 09:00:00,858,/boreholes/view/858/,[],[],DE 02,90.0,,,POINT (10.98714 47.42178)


In [539]:
# subsite_id is not unique, we need view_url as the unique key
gdfSubsiteInfo = gdfSubites.merge(gdfSites,on="view_url")

In [540]:
# consistency check (1)
gdfSubsiteInfo[gdfSubsiteInfo.subsite_type_x != gdfSubsiteInfo.subsite_type_y]

,calm_code,responsible_countries,responsible_person,lithology,subsite_lon,subsite_lat,subsite_elevation,subsite_type_x,created,modified,...,have_data,observation_type,area,site_lon,site_lat,bbox_lat_max,bbox_lat_min,bbox_lon_min,bbox_lon_max,geometry_y


In [541]:
# consistency check (2)
gdfSubsiteInfo[gdfSubsiteInfo.subsite_id_x != gdfSubsiteInfo.subsite_id_y]

,calm_code,responsible_countries,responsible_person,lithology,subsite_lon,subsite_lat,subsite_elevation,subsite_type_x,created,modified,...,have_data,observation_type,area,site_lon,site_lat,bbox_lat_max,bbox_lat_min,bbox_lon_min,bbox_lon_max,geometry_y


In [542]:
# now that we checked consistency, lets get rid of duplicates
gdfSubsiteInfo.rename(columns={"subsite_type_x":"subsite_type",
                              "subsite_id_x":"subsite_id",
                              "geometry_x":"geometry",
                              "geometry_y":"geometry_site",
                              "name":"subsite_name",
                              "site":"site_name"},
                     inplace=True)
del gdfSubsiteInfo["subsite_type_y"]
del gdfSubsiteInfo["subsite_id_y"]
gdfSubsiteInfo = gpd.GeoDataFrame(gdfSubsiteInfo).set_geometry('geometry').set_crs(epsg=4326)
gdfSubsiteInfo

,calm_code,responsible_countries,responsible_person,lithology,subsite_lon,subsite_lat,subsite_elevation,subsite_type,created,modified,...,have_data,observation_type,area,site_lon,site_lat,bbox_lat_max,bbox_lat_min,bbox_lon_min,bbox_lon_max,geometry_site
0,U31 A,"United States,",Nikolay Shiklomanov,Organic Layer thikness:NA; mineral texture -- ...,-148.682100,69.696900,114.259277,activelayer,2014-02-03 11:00:00,2016-10-24 12:00:00,...,False,"CALM, TSP",1934.328,-147.544117,69.719608,69.833333,69.605883,-148.750000,-146.338233,POINT (-147.54412 69.71961)
1,U8,"United States,",Vladimir E. Romanovsky,The organic layer thickness is 0.23 m,-148.716667,69.683333,120.003014,activelayer,2013-12-27 15:00:00,2014-02-03 14:00:00,...,True,"CALM, TSP",1934.328,-147.544117,69.719608,69.833333,69.605883,-148.750000,-146.338233,POINT (-147.54412 69.71961)
2,,"United States,",Gary D. Clow,,-146.338233,69.605883,269.993164,temperatures,2014-08-20 11:00:00,2016-06-02 16:00:00,...,False,"CALM, TSP",1934.328,-147.544117,69.719608,69.833333,69.605883,-148.750000,-146.338233,POINT (-147.54412 69.71961)
3,,"United States,",Doug L. Kane,,-148.750000,69.833333,76.467224,temperatures,2015-10-06 15:00:00,2015-10-06 15:00:00,...,False,"CALM, TSP",1934.328,-147.544117,69.719608,69.833333,69.605883,-148.750000,-146.338233,POINT (-147.54412 69.71961)
4,,"United States,",Vladimir E. Romanovsky,,-148.720766,69.674140,122.868607,temperatures,2013-12-09 14:00:00,2014-02-27 10:00:00,...,True,"CALM, TSP",1934.328,-147.544117,69.719608,69.833333,69.605883,-148.750000,-146.338233,POINT (-147.54412 69.71961)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628,,"United States,",Kenji Yoshikawa,,-163.413000,64.682000,14.000000,temperatures,2014-01-20 13:00:00,2014-06-06 09:00:00,...,False,TSP,0.000,-163.413000,64.682000,64.682000,64.682000,-163.413000,-163.413000,POINT (-163.41300 64.68200)
1629,,"Italy,",Mauro Guglielmin,,10.308250,46.392250,2570.000000,temperatures,2014-01-22 10:00:00,2014-01-22 10:00:00,...,False,TSP,0.000,10.308250,46.392250,46.392250,46.392250,10.308250,10.308250,POINT (10.30825 46.39225)
1630,,"Germany,",Andreas Poschinger,,10.987108,47.421789,2922.000000,temperatures,2014-01-21 15:00:00,2016-10-22 09:00:00,...,False,TSP,0.000,10.983662,47.420216,47.421789,47.418644,10.980181,10.987142,POINT (10.98366 47.42022)
1631,,"Germany,",Andreas Poschinger,,10.987142,47.421778,2922.000000,temperatures,2014-01-21 15:00:00,2016-10-22 09:00:00,...,False,TSP,0.000,10.983662,47.420216,47.421789,47.418644,10.980181,10.987142,POINT (10.98366 47.42022)


In [544]:
del gdfSubsiteInfo["geometry_site"]

In [545]:
gdfSubsiteInfo.to_postgis("t_permafrost_subsites",con=conn,
                          dtype={"calm_code":sqlalchemy.types.VARCHAR(20),
                                 "responsible_countries":sqlalchemy.types.VARCHAR(60),
                                 "responsible_person":sqlalchemy.types.VARCHAR(100),
                                 "lithology":sqlalchemy.types.VARCHAR(),
                                 "subsite_type":sqlalchemy.types.VARCHAR(20),
                                 "created":sqlalchemy.types.TIMESTAMP,
                                 "modified":sqlalchemy.types.TIMESTAMP,
                                 "subsite_id":sqlalchemy.types.INT,
                                 "view_url":sqlalchemy.types.VARCHAR(100),
                                 "entries":sqlalchemy.types.VARCHAR(5000),
                                 "references":sqlalchemy.types.VARCHAR(1500),
                                 "gtn_p":sqlalchemy.types.VARCHAR(20),
                                 "drilling_method":sqlalchemy.types.VARCHAR(200),
                                 "morphology":sqlalchemy.types.VARCHAR(1000),
                                 "":sqlalchemy.types.VARCHAR(),
                                 "subsite_name":sqlalchemy.types.VARCHAR(100),
                                 "site_name":sqlalchemy.types.VARCHAR(100),
                                 "site_id":sqlalchemy.types.INT,
                                 "site_url":sqlalchemy.types.VARCHAR(100),
                                 "country":sqlalchemy.types.VARCHAR(40),
                                 "type":sqlalchemy.types.VARCHAR(20),
                                 "code":sqlalchemy.types.VARCHAR(20),
                                 "vegetation":sqlalchemy.types.VARCHAR(50),
                                 "permafrost":sqlalchemy.types.VARCHAR(50),
                                 "iso2":sqlalchemy.types.VARCHAR(2),
                                 "iso3":sqlalchemy.types.VARCHAR(3),
                                 "country_official_name":sqlalchemy.types.VARCHAR(80),
                                 "observation_type":sqlalchemy.types.VARCHAR(20),
                                 },if_exists="replace")

In [546]:
#gdfSubsiteInfo = gpd.read_postgis("SELECT * FROM t_permafrost_subsites",con=conn,geom_col="geometry")

## Download Active Layer (Thawing) Data

As the subsite ID's are not unique between activelayer and borehole temperature sites, we need two loops. We don't bother to store the zip files but download data into a buffer and extract the content from there.

In [556]:
Path("./sun/activelayers/data").mkdir(exist_ok=True,parents=True)

ok = []
ko = []
already_had = 0

count = 0
for subsite_id in tqdm(gdfSubsiteInfo[(gdfSubsiteInfo.have_data)&(gdfSubsiteInfo.subsite_type=="activelayer")].subsite_id.values):
    count += 1
    url = "http://gtnpdatabase.org/rest/activelayers/dlpackage/{}/true".format(subsite_id)
    Path("./sun/activelayers/data/{}".format(subsite_id)).mkdir(exist_ok=True,parents=True)
    if len(os.listdir("./sun/activelayers/data/{}".format(subsite_id))) > 0:
        already_had += 1
        continue
    r = requests.get(url)
    if r.ok:
        zf = zipfile.ZipFile(io.BytesIO(r.content))
        zf.extractall("./sun/activelayers/data/{}".format(subsite_id))
        ok.append(subsite_id)
    else:
        ko.append(subsite_id)
    time.sleep(1)
print("Looked for {} records (already had {}), received {}. {} errors ({})".format(count-already_had,already_had,
                                                                  len(ok),len(ko),ko))

100%|██████████| 104/104 [00:00<00:00, 63356.23it/s]

Looked for 0 records (already had 104), received 0. 0 errors ([])


### Data Cleansing

The zip files contain a variety of records, many of which we don't need as we already acquired the metadata. Any observations are logged. Note that we now start ground-up as there may be more files in the zip file than subsites.

In [587]:
# taken from https://naysan.ca/2020/05/09/pandas-to-postgresql-using-psycopg2-bulk-insert-performance-benchmark/
# Many thanks, what a cool idea!
def copy_from_stringio(conn, df, table):
    """
    Here we are going save the dataframe in memory 
    and use copy_from() to copy it to the table
    """
    buffer = io.StringIO()
    df.to_csv(buffer, index=False, header=False)
    buffer.seek(0)
    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",", columns=df.columns)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    #print("copy_from_stringio() done")
    cursor.close()

In [599]:
def clean_save_activelayer(df,dataset_id,conn):
    dfOffsets = df[["offset_x","offset_y"]].copy()
    dfOffsets["idx"] = dfOffsets.index

    del df["offset_x"]
    del df["offset_y"]

    df = df.transpose()
    df.index = pd.to_datetime(df.index)
    df.index.name = None

    ddf = df.stack().reset_index().rename(columns={"level_0":"datetime_date","level_1":"idx",0:"depth_m"}).merge(dfOffsets,on="idx")
    ddf.index = ddf.datetime_date
    ddf.index.name = None
    del ddf["idx"]
    ddf["dataset_id"] = dataset_id
    ddf["subsite_type"] = "activelayer"
    
    ddf.dropna(subset=["depth_m"],inplace=True)
    ddf.depth_m = ddf.depth_m/100. # convert cm to m
    
    try:
        conn.execute("DELETE FROM t_permafrost_depth_data WHERE dataset_id={} AND subsite_type='activelayer'".format(dataset_id))
    except:
        pass
    copy_from_stringio(conn.raw_connection(),ddf,"t_permafrost_depth_data")

    return ddf

In [600]:
conn.execute("""CREATE TABLE IF NOT EXISTS t_permafrost_depth_data (
	"datetime_date" timestamp(6), 
	"depth_m" real, 
	"offset_x" real, 
	"offset_y" real, 
	"dataset_id" integer, 
	"subsite_type" varchar(20)
);""")
conn.execute("COMMENT ON COLUMN t_permafrost_depth_data.datetime_date is 'Timestamp of observation';")
conn.execute("COMMENT ON COLUMN t_permafrost_depth_data.depth_m is 'Depth in [m] where ground was frozen';")
conn.execute("COMMENT ON COLUMN t_permafrost_depth_data.offset_x is 'x offset in [m] of borehole grid';")
conn.execute("COMMENT ON COLUMN t_permafrost_depth_data.offset_y is 'y offset in [m] of borehole grid';")
conn.execute("COMMENT ON COLUMN t_permafrost_depth_data.dataset_id is 'Dataset ID, unique for activelayer data only';")
conn.execute("COMMENT ON COLUMN t_permafrost_depth_data.subsite_type is 'set to activelayer';")

In [601]:
catalogue = []

for r,dd,ff in os.walk("./sun/activelayers/"):
    if ".ipynb_checkpoints" in r or "view" in r:
        continue
    for f in ff:
        if f == "index.html":
            continue
        bits = os.path.splitext(f)
        subsite_id = r.split("/")[-1]
        data = {}
        from_dt = None
        to_dt = None
        num_records = -1
        num_years = 0
        avg_measurements_year = 0
        if f in ["GTNP_Citation_ALT.txt","GTNP_ISO_19115_2.xml"]:
            # these are generic text documents, which are identical everywhere
            continue
        elif bits[1] == ".parquet":
            continue
        elif bits[1] == ".txt":
            # this is a metadata text document
            continue
        elif bits[1] == ".csv" and not "moisture" in f:
            try:
                dfData = pd.read_csv(os.path.join(r,f)).replace(-999.0,np.nan)
            except Exception as e:
                log_finding({"topic":"active layer depths",
                            "subsite_id":subsite_id,
                            "filename":f,
                            "diagnosis":"{}".format(sys.exc_info()).strip(),
                            "fix":"skipping file",
                            "needs_attention":True})
                continue
            
            if isinstance(dfData.index, pd.MultiIndex): # This was a weird error in one file which did not raise an exception
                log_finding({"topic":"active layer depths",
                            "subsite_id":subsite_id,
                            "filename":f,
                            "diagnosis":"something is not right in this file, partially more data column(s) than columns result in a MultiIndex",
                            "fix":"skipping file",
                            "needs_attention":True})
                continue
                
            dfData = clean_save_activelayer(dfData,subsite_id,conn)
            from_dt = dfData.datetime_date.min()
            to_dt = dfData.datetime_date.max()
            num_records = len(dfData)
            dfByYear = dfData.groupby(pd.Grouper(freq="Y")).size().replace(0,np.nan).dropna()
            num_years = len(dfByYear)
            avg_measurements_year = dfByYear.quantile(0.5)
            
            dataset = f.split("-")[2].replace("Dataset_","")
            parameter = "depth_m"
        elif bits[1] == ".csv" and "moisture" in f:
            dataset = f.split("-")[2].replace("Dataset_","")
            log_finding({"topic":"active layer depths",
                         "dataset":dataset,
                        "subsite_id":subsite_id,
                        "site":f.split("-")[1],
                        "filename":f,
                        "diagnosis":"dataset contains moisture, not temperature, data",
                        "fix":"ignored",
                        "needs_attention":False})
            continue
        else:
            print("Unknown File {}".format(os.path.join(r,f)))
            continue
        catalogue.append({"filename":f,"extension":bits[1],"subsite_id":subsite_id,"dataset":dataset,"from_dt":from_dt,
                          "to_dt":to_dt,"num_records":num_records,"parameter":parameter,"path":os.path.join(r,f),
                          "site":f.split("-")[1],"num_years":num_years,"avg_measurements_year":avg_measurements_year,
                          "subsite_type":"activelayer"})
                              
dfCatalogue = pd.DataFrame(catalogue)
dfCatalogue.subsite_id = pd.to_numeric(dfCatalogue.subsite_id,errors="coerce")
dfCatalogue.dropna(axis=0,subset=["subsite_id"],inplace=True)
dfCatalogue

,filename,extension,subsite_id,dataset,from_dt,to_dt,num_records,parameter,path,site,num_years,avg_measurements_year,subsite_type
0,Activelayer_13-West_Dock_1ha-Dataset_276-Spora...,.csv,13,276,1996-09-01,2019-08-22,2827,depth_m,./sun/activelayers/data/13/Activelayer_13-West...,West_Dock_1ha,24,121.0,activelayer
1,Activelayer_88-Akhmelo_Channel-Dataset_255-Spo...,.csv,88,255,1996-09-06,2019-09-01,2783,depth_m,./sun/activelayers/data/88/Activelayer_88-Akhm...,Akhmelo_Channel,22,121.0,activelayer
2,Activelayer_118-Umbozero-Dataset_243-Sporadic-...,.csv,118,243,2010-08-24,2016-08-22,968,depth_m,./sun/activelayers/data/118/Activelayer_118-Um...,Umbozero,7,121.0,activelayer
3,Activelayer_78-Andryushkino-Dataset_266-Sporad...,.csv,78,266,2005-09-22,2019-09-24,2492,depth_m,./sun/activelayers/data/78/Activelayer_78-Andr...,Andryushkino,15,121.0,activelayer
4,Activelayer_1-Happy_Valley_1km-Dataset_13-Spor...,.csv,1,13,1995-07-08,2019-08-12,4390,depth_m,./sun/activelayers/data/1/Activelayer_1-Happy_...,Happy_Valley_1km,25,117.0,activelayer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Activelayer_49-UNISCALM-Dataset_1505-Sporadic-...,.csv,49,1505,2011-08-23,2018-08-17,968,depth_m,./sun/activelayers/data/49/Activelayer_49-UNIS...,UNISCALM,8,121.0,activelayer
88,Activelayer_110-Yubileynoe_2_WET-Dataset_271-S...,.csv,110,271,2007-07-08,2007-07-08,121,depth_m,./sun/activelayers/data/110/Activelayer_110-Yu...,Yubileynoe_2_WET,1,121.0,activelayer
89,Activelayer_249-Urengoy_GAS_FIELD_GP15-Dataset...,.csv,249,1523,2008-09-04,2019-08-22,1200,depth_m,./sun/activelayers/data/249/Activelayer_249-Ur...,Urengoy_GAS_FIELD_GP15,12,102.0,activelayer
90,Activelayer_30-Lousy_Point__Grid_-Dataset_228-...,.csv,30,228,1998-08-20,2008-08-21,1331,depth_m,./sun/activelayers/data/30/Activelayer_30-Lous...,Lousy_Point__Grid_,11,121.0,activelayer


In [602]:
dfCatalogue.to_parquet("./sun/activelayers/datasets.parquet")

## Download Active Layer Temperature Data

This dataset often contains more than one dataset per folder as there also exist e.g. satellite surface temperatures. These can be acquired via https://modis.gsfc.nasa.gov/data/, which may undergo data product updates and is suggested as the ground truth reference dataset.

In [604]:
Path("./sun/boreholes/data").mkdir(exist_ok=True,parents=True)

ok = []
ko = []
already_had = 0

count = 0
for subsite_id in tqdm(gdfSubsiteInfo[(gdfSubsiteInfo.have_data)&(gdfSubsiteInfo.subsite_type=="temperatures")].subsite_id.values):
    count += 1
    url = "http://gtnpdatabase.org/rest/boreholes/dlpackage/{}/true".format(subsite_id)
    Path("./sun/boreholes/data/{}".format(subsite_id)).mkdir(exist_ok=True,parents=True)
    if len(os.listdir("./sun/boreholes/data/{}".format(subsite_id))) > 0:
        already_had += 1
        continue
    r = requests.get(url)
    if r.ok:
        zf = zipfile.ZipFile(io.BytesIO(r.content))
        zf.extractall("./sun/boreholes/data/{}".format(subsite_id))
        ok.append(subsite_id)
    else:
        ko.append(subsite_id)
    time.sleep(0.25)
print("Looked for {} records (already had {}), received {}. {} errors ({})".format(count-already_had,already_had,
                                                                  len(ok),len(ko),ko))

100%|██████████| 498/498 [00:00<00:00, 58430.22it/s]

Looked for 0 records (already had 498), received 0. 0 errors ([])


In [605]:
alldata = []

for r,dd,ff in os.walk("./sun/boreholes/data"):
    if ".ipynb_checkpoints" in r:
        continue
    for f in ff:
        if f.endswith(".csv"):
            part_1,part_2 = re.split("-Dataset_",f)
            
            fields = part_2.split("-")
            entry = dict(zip(["field_{}".format(i) for i in range(len(fields))],fields))
            entry["subsite_id"]=int(part_1.split("-")[0].replace("Borehole_",""))
            entry["filename"] = f
            entry["path"] = os.path.join(r,f)
            entry["extension"] = ".csv"
            
            if entry["field_3"] == "Ground_Temperature":
                alldata.append(entry)
            else:
                entry = {}
                
dfCatalogueTemps = pd.DataFrame(alldata).rename(columns={"field_0":"dataset",
                                                  "field_1":"aggregation",
                                                  "field_2":"aggregation_frequency",
                                                  "field_3":"parameter",
                                                  "field_4":"instrument"})
dfCatalogueTemps.instrument = dfCatalogueTemps.instrument.str.replace(".timeserie.csv","")
dfCatalogueTemps

,dataset,aggregation,aggregation_frequency,parameter,instrument,subsite_id,filename,path,extension
0,1923,Average,Monthly,Ground_Temperature,Unknown,1680,Borehole_1680-Komsomolskaya-Dataset_1923-Avera...,./sun/boreholes/data/1680/Borehole_1680-Komsom...,.csv
1,1874,Average,Monthly,Ground_Temperature,Unknown,1631,Borehole_1631-Ivdel-Dataset_1874-Average-Month...,./sun/boreholes/data/1631/Borehole_1631-Ivdel-...,.csv
2,656,Constant_Over_Interval,Daily,Ground_Temperature,Thermistor_Automated,88,Borehole_88-Deadhorse_2__new_instrumentation_-...,./sun/boreholes/data/88/Borehole_88-Deadhorse_...,.csv
3,660,Constant_Over_Interval,Daily,Ground_Temperature,Thermistor_Automated,106,Borehole_106-Franklin_Bluffs__surface_-Dataset...,./sun/boreholes/data/106/Borehole_106-Franklin...,.csv
4,1906,Average,Monthly,Ground_Temperature,Unknown,1663,Borehole_1663-Taiga-Dataset_1906-Average-Month...,./sun/boreholes/data/1663/Borehole_1663-Taiga-...,.csv
...,...,...,...,...,...,...,...,...,...
429,2074,Average,Annually,Ground_Temperature,Thermistor_Automated,137,Borehole_137-Norris_Creek_NC-01-Dataset_2074-A...,./sun/boreholes/data/137/Borehole_137-Norris_C...,.csv
430,1930,Average,Monthly,Ground_Temperature,Unknown,1687,Borehole_1687-Tolstovka__Amurskaya_agrexpst_-D...,./sun/boreholes/data/1687/Borehole_1687-Tolsto...,.csv
431,1535,Constant_Over_Interval,2_hours,Ground_Temperature,Thermistor_Automated,1185,Borehole_1185-Rhonda_Basin-Dataset_1535-Consta...,./sun/boreholes/data/1185/Borehole_1185-Rhonda...,.csv
432,1859,Average,Monthly,Ground_Temperature,Unknown,1616,Borehole_1616-Sterlitamak-Dataset_1859-Average...,./sun/boreholes/data/1616/Borehole_1616-Sterli...,.csv


In [607]:
dfCatalogueTemps.to_parquet("./sun/boreholes/datasets.parquet")

### Data Cleansing

In [612]:
def compute_actual_data_frequency(dfData):
    datetime_date = pd.Series(dfData.index)
    offset = abs((datetime_date-datetime_date.shift(-1)).dt.total_seconds().median())/86400.
    if (350. <= offset) and (offset <= 385.):
        aggregation_frequency = "Annually"
        expected_frequency = 365.25*24*3600
        resample_alias = "Y"
    elif (90. <= offset) and (offset <= 92.):
        aggregation_frequency = "Quarterly"
        resample_alias = "3M"
        expected_frequency = np.mean([31+28.25+31,30+31+30,31+31+30,31+30+31])*24*3600
    elif (30. <= offset) and (offset <= 31.1):
        aggregation_frequency = "Monthly"
        expected_frequency = np.mean([31,28.25,31,30,31,30,31,31,30,31,30,31])*24*3600
        resample_alias = "M"
    elif (.99 <= offset) and (offset <= 1.01):
        aggregation_frequency = "Daily"
        expected_frequency = 24*3600
        resample_alias = "D"
    elif (.49 <= offset) and (offset <= .51):
        aggregation_frequency = "12_hours"
        expected_frequency = 12*3600
        resample_alias = "12H"
    elif (.24 <= offset) and (offset <= .26):
        aggregation_frequency = "6_hours"
        expected_frequency = 6*3600
        resample_alias = "6H"
    elif (.16 <= offset) and (offset <= .17):
        aggregation_frequency = "4_hours"
        expected_frequency = 2*3600
        resample_alias = "4H"
    elif (.12 <= offset) and (offset <= .13):
        aggregation_frequency = "3_hours"
        expected_frequency = 3*3600
        resample_alias = "3H"
    elif (.08 <= offset) and (offset <= .085):
        aggregation_frequency = "2_hours"
        expected_frequency = 2*3600
        resample_alias = "2H"
    elif (.04 <= offset) and (offset <= .045):
        aggregation_frequency = "Hourly"
        expected_frequency = 1*3600
        resample_alias = "H"
    else:
        aggregation_frequency = "TBD"
        print("WARNING")
        expected_frequency = 0.
        resample_alias = ""

    return {"aggregation_frequency":aggregation_frequency,"expected_frequency":expected_frequency,"resample_alias":resample_alias,"offset":offset}




def clean_save_temperatures(df,dataset_id,conn):
    for c in df.columns:
        if "Date" in c:
            continue
        else:
            value_columns.append(c)
    df[value_columns] = df[value_columns].apply(lambda x: np.where(x < -273.15,np.nan,x))
    
    df.index = pd.to_datetime(df["Date/Depth"])
    df.index.name = None
    del df["Date/Depth"]
    
    frequencies = compute_actual_data_frequency(df)
    if frequencies["aggregation_frequency"] == "TBD":
        log_finding({"topic":"borehole temperatures",
                     "dataset_id":dataset_id,
                     "diagnosis":"dataset with irregular time resolution which cannot be resolved.",
                     "fix":"Skip dataset and ignoring {} records.".format(len(df)),
                     "needs_attention":True
                    })
        return pd.DataFrame()
    coverage = (df.index.max()-df.index.min()).total_seconds()
    expected_num_records = coverage/frequencies["expected_frequency"]+1
    actual_num_records = len(df)
    completeness = (actual_num_records)/expected_num_records
    
    if completeness > 1.001:
        df = df.resample(frequencies["resample_alias"]).mean()
        log_finding({"topic":"borehole temperatures",
                     "dataset_id":dataset_id,
                     "diagnosis":"duplicate datetimes",
                     "fix":"resampled dataframe to aggregation frequency {}(pandas: {}). Old number of records {}, new number of records {}.".format(frequencies['aggregation_frequency'],
                                                                                                                                                     frequencies['resample_alias'],
                                                                                                                                                     actual_num_records,
                                                                                                                                                     len(df)),
                     "needs_attention":True
                    })
        new_completeness = (len(df))/expected_num_records
        #print(dataset_id,actual_num_records,completeness,frequencies,len(df),new_completeness)
        #assert False

    df = df.apply(lambda x: np.where(x < -273.15,np.nan,x)).dropna(axis=0,how="all")
    ddf = df.stack().reset_index().rename(columns={"level_0":"datetime_date","level_1":"depth_m",0:"temperature_degC"})
    ddf.depth_m = pd.to_numeric(ddf.depth_m)
    ddf.temperature_degC = pd.to_numeric(ddf.temperature_degC)
    ddf.dropna(axis=0,subset=["temperature_degC"],inplace=True)
    
    ddf["dataset_id"] = dataset_id
    
    ddf.index = ddf.datetime_date
    ddf.index.name = None    
    
    return ddf

    try:
        conn.execute("DELETE FROM t_permafrost_temperature_data WHERE dataset_id={}".format(dataset_id))
    except:
        pass
    
    #ddf.to_sql("t_permafrost_temperature_data",con=conn,if_exists="append",dtype={"datetime_date":sqlalchemy.types.TIMESTAMP,
    #                                                                              "depth_m":sqlalchemy.types.REAL,
    #                                                                              "temperature_degC":sqlalchemy.types.REAL,
    #                                                                              "dataset_id":sqlalchemy.types.INTEGER},
    #          index=False)
    copy_from_stringio(conn.raw_connection(),ddf,"t_permafrost_temperature_data")
    
    return ddf

In [614]:
#column_names = set()
alldata = []
for i,r in dfCatalogueTemps.iterrows():
    
    path = r.path
    ddf = pd.read_csv(path)
    value_columns = []
    if len(ddf) <= 0:
        #print("Already Empty {}".format(path))
        log_finding({"topic":"borehole temperatures",
                     "dataset":r.dataset,
                    "subsite_id":r.subsite_id,
                    "filename":r.filename,
                    "diagnosis":"file contains no data",
                    "fix":"ignored",
                    "needs_attention":True})
        continue
        
    dfData = clean_save_temperatures(ddf,r.dataset,conn)
    #continue
    if len(dfData.columns) <= 0:
        log_finding({"topic":"borehole temperatures",
                     "dataset":r.dataset,
                    "subsite_id":r.subsite_id,
                    "filename":r.filename,
                    "diagnosis":"file contains unknown time resolution or too many gaps",
                    "fix":"ignored, see previous message",
                    "needs_attention":True})
        continue
    elif len(dfData) <= 0:
        #print("Made Empty {}".format(path))
        log_finding({"topic":"borehole temperatures",
                     "dataset":r.dataset,
                    "subsite_id":r.subsite_id,
                    "filename":r.filename,
                    "diagnosis":"file contains no valid data, only entries less than -273.15 deg C",
                    "fix":"ignored",
                    "needs_attention":True})
        continue
    
    from_dt = dfData.datetime_date.min()
    to_dt = dfData.datetime_date.max()
    num_records = len(dfData)
    dfByYear = dfData.groupby(pd.Grouper(freq="Y")).size().replace(0,np.nan).dropna()
    num_years = len(dfByYear)
    avg_measurements_year = dfByYear.quantile(0.5)

    parameter = "temperature_degC"

    alldata.append({"filename":r.filename,"extension":r.extension,"subsite_id":r.subsite_id,"dataset":r.dataset,"from_dt":from_dt,
                          "to_dt":to_dt,"num_records":num_records,"parameter":parameter,"path":r.path,
                          "site":r.filename.split("-")[1],"num_years":num_years,"avg_measurements_year":avg_measurements_year,
                          "subsite_type":"temperatures"})
#column_names
pd.DataFrame(alldata)

WARNING
WARNING
WARNING
WARNING


/home/anaconda/anaconda3/envs/arctic/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


WARNING


/home/anaconda/anaconda3/envs/arctic/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


WARNING
WARNING
WARNING
WARNING
WARNING
WARNING
WARNING
WARNING
WARNING


/home/anaconda/anaconda3/envs/arctic/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


WARNING
WARNING
WARNING
WARNING
WARNING
WARNING
WARNING
WARNING
WARNING
WARNING
WARNING


,filename,extension,subsite_id,dataset,from_dt,to_dt,num_records,parameter,path,site,num_years,avg_measurements_year,subsite_type
0,Borehole_1680-Komsomolskaya-Dataset_1923-Avera...,.csv,1680,1923,1963-09-15 00:00:00,1965-12-15 12:00:00,194,temperature_degC,./sun/boreholes/data/1680/Borehole_1680-Komsom...,Komsomolskaya,3,81.0,temperatures
1,Borehole_1631-Ivdel-Dataset_1874-Average-Month...,.csv,1631,1874,1938-01-15 12:00:00,1990-12-15 12:00:00,2042,temperature_degC,./sun/boreholes/data/1631/Borehole_1631-Ivdel-...,Ivdel,50,49.0,temperatures
2,Borehole_88-Deadhorse_2__new_instrumentation_-...,.csv,88,656,2008-06-01 00:00:00,2013-05-31 00:00:00,24536,temperature_degC,./sun/boreholes/data/88/Borehole_88-Deadhorse_...,Deadhorse_2__new_instrumentation_,6,4516.5,temperatures
3,Borehole_106-Franklin_Bluffs__surface_-Dataset...,.csv,106,660,2001-06-01 00:00:00,2013-05-31 00:00:00,48736,temperature_degC,./sun/boreholes/data/106/Borehole_106-Franklin...,Franklin_Bluffs__surface_,13,4015.0,temperatures
4,Borehole_1663-Taiga-Dataset_1906-Average-Month...,.csv,1663,1906,1930-01-15 12:00:00,1990-12-15 12:00:00,2124,temperature_degC,./sun/boreholes/data/1663/Borehole_1663-Taiga-...,Taiga,60,38.5,temperatures
...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,Borehole_137-Norris_Creek_NC-01-Dataset_2074-A...,.csv,137,2074,2007-07-02 00:00:00,2014-07-02 00:00:00,8,temperature_degC,./sun/boreholes/data/137/Borehole_137-Norris_C...,Norris_Creek_NC,8,1.0,temperatures
403,Borehole_1687-Tolstovka__Amurskaya_agrexpst_-D...,.csv,1687,1930,1951-01-15 12:00:00,1990-12-15 12:00:00,3343,temperature_degC,./sun/boreholes/data/1687/Borehole_1687-Tolsto...,Tolstovka__Amurskaya_agrexpst_,40,83.5,temperatures
404,Borehole_1185-Rhonda_Basin-Dataset_1535-Consta...,.csv,1185,1535,2011-06-20 16:00:00,2012-07-19 14:00:00,15215,temperature_degC,./sun/boreholes/data/1185/Borehole_1185-Rhonda...,Rhonda_Basin,2,7607.5,temperatures
405,Borehole_1616-Sterlitamak-Dataset_1859-Average...,.csv,1616,1859,1933-07-15 12:00:00,1990-12-15 12:00:00,2807,temperature_degC,./sun/boreholes/data/1616/Borehole_1616-Sterli...,Sterlitamak,57,51.0,temperatures


In [615]:
dfCatalogueTemps = dfCatalogueTemps.merge(pd.DataFrame(alldata),on="path")
dfCatalogueTemps.to_parquet("./sun/boreholes/datasets.parquet")

## Compute Summer Active Layer Depths

In [616]:
datasets = pd.read_sql("SELECT DISTINCT dataset_id FROM t_permafrost_temperature_data;",con=conn).dataset_id.values

In [618]:
alldfs = []
for dataset in tqdm(datasets):
    query = """SELECT datetime_date,date_part('year'::text, t_permafrost_temperature_data.datetime_date) AS year,
    date_part('month'::text, t_permafrost_temperature_data.datetime_date) AS month,
    depth_m, "temperature_degC"
    FROM t_permafrost_temperature_data 
    WHERE dataset_id={} AND date_part('month'::text, t_permafrost_temperature_data.datetime_date) in (6,7,8,9)""".format(dataset)

    # WHERE dataset_id={}""".format(dataset)
    dfData = pd.read_sql(query,con=conn)

    datetime_dates = []
    zerocrossings = []

    alldata = []

    dt_slices = dfData.datetime_date.unique()
    for dt_slice in dt_slices:
        ddf = dfData[dfData.datetime_date == dt_slice]
        values = ddf.temperature_degC.values
        if values.min() > 0 or values.max() < 0:
            continue
            zerocrossing = np.nan
        else:
            depths = ddf.depth_m.values
            #print(values,depths)
            f = interp1d(values,depths) #pd.to_numeric(values.index))
            zerocrossing = f(0.)[()]
        alldata.append({"datetime_date":dt_slice,"depth_m":zerocrossing,"dataset_id":dataset,
                        "year":ddf.year.unique()[0],"month":ddf.month.unique()[0]})
    ddf = pd.DataFrame(alldata).dropna()
    
    alldfs.append(ddf)
    
dfActiveLayerDepthsFromTemps = pd.DataFrame().append(alldfs)
dfActiveLayerDepthsFromTemps

100%|██████████| 432/432 [10:52<00:00,  1.51s/it]


,datetime_date,depth_m,dataset_id,year,month
0,2006-09-23,1.881356,1145,2006.0,9.0
1,2006-09-28,1.881356,1145,2006.0,9.0
2,2006-09-29,1.879310,1145,2006.0,9.0
3,2006-09-30,1.877193,1145,2006.0,9.0
4,2006-09-24,1.883333,1145,2006.0,9.0
...,...,...,...,...,...
328,2011-09-27,2.617021,625,2011.0,9.0
329,2011-09-28,2.647059,625,2011.0,9.0
330,2011-09-29,2.660377,625,2011.0,9.0
331,2011-09-30,2.636364,625,2011.0,9.0


In [627]:
dfActiveLayerDepthsFromTempsReduced = dfActiveLayerDepthsFromTemps.groupby(["dataset_id","year"]).quantile(0.5)[["datetime_date","depth_m"]].reset_index()
del dfActiveLayerDepthsFromTempsReduced["year"]
dfActiveLayerDepthsFromTempsReduced["subsite_type"] = "temperatures"
dfActiveLayerDepthsFromTempsReduced

,dataset_id,datetime_date,depth_m,subsite_type
0,29,2008-09-27 03:00:00,2.512195,temperatures
1,29,2009-07-31 15:00:00,2.272478,temperatures
2,29,2010-07-31 21:00:00,2.438928,temperatures
3,29,2011-07-31 21:00:00,2.469474,temperatures
4,29,2012-08-01 03:00:00,2.456808,temperatures
...,...,...,...,...
1750,2120,2018-09-09 00:00:00,0.413220,temperatures
1751,2120,2019-09-20 00:00:00,0.219266,temperatures
1752,2122,2017-09-16 00:00:00,0.933824,temperatures
1753,2122,2018-07-17 00:00:00,0.722364,temperatures


In [629]:
conn.execute("DELETE FROM t_permafrost_depth_data WHERE subsite_type='temperatures'")
copy_from_stringio(conn.raw_connection(), dfActiveLayerDepthsFromTempsReduced, "t_permafrost_depth_data")

In [ ]:
dfActiveLayerDepthsFromTempsReduced